# LRT Request Analysis

## Functionalities
- Analysis of RPCs and queries of LRT requests.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
    ...
    loadgen.tar.gz
```
`*_service*.tar.gz` and `apigateway*.tar.gz` tarballs contain RPC log files named `calls.log` and database query log files named `queries.log`. A tarball `loadgen.tar.gz` contains a request log file named `loadgen.log`.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-11-03-23-37-35"

## Notebook Setup

In [ ]:
# Import libraries.
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import tarfile
import warnings
warnings.filterwarnings("ignore")

# Constants
REQUEST_LOG_PATTERN = r"^\[(\d+\-\d+\-\d+ \d+:\d+:\d+.\d+)\] (.+) (.+) (\d+) - latency=(\d+.\d+)$"
URL_PATTERN = r"^http://[\w\.]+:\d+/{path}/?\??{qs}$"
REQUEST_TO_TYPE = {
    (URL_PATTERN.format(path="account/\d+", qs=""), "GET"): "retrieve_account",
    (URL_PATTERN.format(path="account", qs=""), "POST"): "create_account",
    (URL_PATTERN.format(path="account/\d+", qs=""), "PUT"): "update_account",
    (URL_PATTERN.format(path="follow", qs="followee_id=\d+"), "GET"): "retrieve_account_followers",
    (URL_PATTERN.format(path="follow", qs="follower_id=\d+"), "GET"): "retrieve_account_followees",
    (URL_PATTERN.format(path="follow", qs=""), "POST"): "follow_account",
    (URL_PATTERN.format(path="follow/\d+", qs=""), "DELETE"): "delete_follow",
    (URL_PATTERN.format(path="like", qs="account_id=\d+"), "GET"): "retrieve_account_likes",
    (URL_PATTERN.format(path="like", qs="post_id=\d+"), "GET"): "retrieve_post_likes",
    (URL_PATTERN.format(path="like", qs=""), "POST"): "like_post",
    (URL_PATTERN.format(path="like/\d+", qs=""), "DELETE"): "delete_like",
    (URL_PATTERN.format(path="post", qs=""), "GET"): "retrieve_recent_posts",
    (URL_PATTERN.format(path="post", qs="author_id=\d+"), "GET"): "retrieve_account_posts",
    (URL_PATTERN.format(path="post/\d+", qs=""), "GET"): "retrieve_post",
    (URL_PATTERN.format(path="post", qs=""), "POST"): "create_post",
    (URL_PATTERN.format(path="post/\d+", qs=""), "DELETE"): "delete_post"
}
COMPONENT_TARBALL_PATTERNS = [
    r"^apigateway.*\.tar\.gz$",
    r"^.+_service.*\.tar\.gz$",
]
RPC_LOG_PATTERN = r"^\[(.+)\] pid=(.+) tid=(.+) request_id=(.+) server=(.+) function=(.+) latency=(.+)$"

## Request Log Parsing

In [ ]:
# Parse logs
requests = {"timestamp": [], "method": [], "url": [], "status_code": [], "latency": [], "request_id": []}
node_names = os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs"))
for node_name in node_names:
  node_min_timestamp = None
  tarball_path = os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name, "loadgen.tar.gz")
  if os.path.exists(tarball_path):
    with tarfile.open(tarball_path, "r:gz") as tar:
      for filename in tar.getnames():
        if filename.endswith("loadgen.log"):
          with tar.extractfile(filename) as requests_log_file:
            for log in requests_log_file:
              timestamp, method, url, status_code, latency = re.match(REQUEST_LOG_PATTERN, log.decode("utf-8")).groups()
              request_id = re.findall("request_id=([a-zA-Z0-9]+)&?", url)[0]
              url = re.sub("limit=\d+&?", "", url)
              url = re.sub("offset=\d+&?", "", url)
              url = re.sub("request_id=[a-zA-Z0-9]+&?", "", url)
              url = re.sub("&$", "", url)
              url = re.sub("\?$", "", url)
              timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")
              if node_min_timestamp is None:
                node_min_timestamp = timestamp
              requests["timestamp"].append((timestamp - node_min_timestamp).total_seconds())
              requests["method"].append(method)
              requests["url"].append(url)
              requests["status_code"].append(int(status_code))
              requests["latency"].append(float(latency))
              requests["request_id"].append(request_id)

In [ ]:
# Build data frame
requests = pd.DataFrame.from_dict(requests)
requests.sort_values(by="timestamp", ascending=True, inplace=True)
requests["type"] = requests.apply(
    lambda r: [request_type for ((pattern, method), request_type) in REQUEST_TO_TYPE.items()
    if method == r["method"] and re.match(pattern, r["url"])][0], axis=1)

## RPC Log Parsing

In [ ]:
# Parse logs
rpc = {"node_name": [], "timestamp": [], "request_id": [], "server": [], "function": [], "latency": []}
node_names = os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs"))
for node_name in node_names:
  for tarball_name in os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name)):
    if sum([1 if re.match(tarball_pattern, tarball_name) else 0 for tarball_pattern in COMPONENT_TARBALL_PATTERNS]):
      tarball_path = os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name, tarball_name)
      with tarfile.open(tarball_path, "r:gz") as tar:
        for filename in tar.getnames():
          if filename.endswith("calls.log"):
            with tar.extractfile(filename) as log_file:
              for row in log_file:
                rpc_log_match = re.match(RPC_LOG_PATTERN, row.decode("utf-8"))
                if rpc_log_match:
                  timestamp, pid, tid, request_id, server, function, latency = rpc_log_match.groups()
                  rpc["node_name"].append(node_name)
                  rpc["timestamp"].append(datetime.datetime.strptime(timestamp[:-3], "%H:%M:%S.%f"))
                  rpc["request_id"].append(request_id)
                  rpc["server"].append(server)
                  rpc["function"].append(function)
                  rpc["latency"].append(float(latency) * 1000)

In [ ]:
# Build data frame
rpc = pd.DataFrame.from_dict(rpc)

# Get values
function_names = sorted(rpc["function"].unique())
min_timestamp = rpc["timestamp"].values.min()

# (Re) Build columns
rpc["timestamp"] = rpc.apply(lambda r: (r["timestamp"] - min_timestamp).total_seconds(), axis=1)
rpc["window"] = rpc.apply(lambda r: int(r["timestamp"]), axis=1)

## Analysis of LRT Requests

In [ ]:
# Analysis of requests with latency over P99.9
lrt_requests = requests[(requests["latency"] > requests[requests["status_code"] == 200]["latency"].quantile(0.999))]
for lrt_request in lrt_requests.to_dict("records"):
    print("Request ID: %s" % lrt_request["request_id"])
    print("  Type: %s" % lrt_request["type"])
    print("  RPCs:")
    for lrt_request_rpc in rpc[(rpc["request_id"] == lrt_request["request_id"])].to_dict("records"):
        print("    %s - %s" % (lrt_request_rpc["function"], lrt_request_rpc["latency"]))